In [39]:
def extract_answer(rationale):
    answer = str(int(rationale.split("\n")[-1].split(",")[1]))
    return answer

def count_c(a_digit, b_digit, c, mode):
    a_digit = int(a_digit)
    b_digit = int(b_digit)
    c = int(c)
    if mode == "add":
        if a_digit + b_digit + c>= 10:
            return 1
        else:
            return 0
    elif mode == "subtract":
        if a_digit - b_digit + c < 0:
            return -1
        else:
            return 0

def count_sub(a_digit, b_digit, c):
    r = a_digit - b_digit + c
    if r < 0:
        return str(10 + r)
    else:
        return str(r)

def gen_cot_rationale(a, b, mode="add"):
    '''
    return the cot rationale for the question
    '''
    a_digits = [digit for digit in str(a)]
    b_digits = [digit for digit in str(b)]
    rationale = ""
    answer = ""
    c = 0
    if mode == "add":
        gt = int(a) + int(b)
        for _ in range(len(str(int(a)+int(b)))+1):
            line = f"{''.join(a_digits)}+{''.join(b_digits)},{answer},C:{c}\n"
            rationale += line
            if a_digits and b_digits:
                answer = str(int(a_digits[-1]) + int(b_digits[-1]) + c)[-1] + answer
                c = count_c(a_digits[-1], b_digits[-1], c, mode)
                a_digits.pop()
                b_digits.pop()
            elif a_digits:
                answer = str(int(a_digits[-1]) + c)[-1] + answer
                c = count_c(a_digits[-1], 0, c, mode)
                a_digits.pop()
            elif b_digits:
                answer = str(int(b_digits[-1]) + c)[-1] + answer
                c = count_c(0, b_digits[-1], c, mode)
                b_digits.pop()
            else:
                if c:
                    answer = str(c) + answer
                c = 0
    elif mode == "subtract":
        gt = int(a) - int(b)
        for _ in range(max(len(str(a)), len(str(b)))+1):
            line = f"{''.join(a_digits)}-{''.join(b_digits)},{answer},C:{c}\n"
            rationale += line
            if a_digits and b_digits:
                answer = count_sub(int(a_digits[-1]),int(b_digits[-1]),c)[-1] + answer
                c = count_c(a_digits[-1], b_digits[-1], c, mode)
                a_digits.pop()
                b_digits.pop()
            elif a_digits:
                answer = count_sub(int(a_digits[-1]),0,c)[-1] + answer
                c = count_c(a_digits[-1], 0, c, mode)
                a_digits.pop()
            elif b_digits:
                answer = count_sub(0,int(b_digits[-1]),c)[-1] + answer
                c = count_c(0, b_digits[-1], c, mode)
                b_digits.pop()
            else:
                if c:
                    answer = count_sub(0,0,c) + answer
                c = 0
    rationale = rationale.strip()
    assert int(extract_answer(rationale)) == int(gt)
    return f"{rationale}\n{gt}"

def gen_mod_add_cot_rationale(a, b, P):
    rationale = gen_cot_rationale(a, b, mode="add")
    if a + b >= P:
        rationale += f"\n{a+b}>={P}\n"
        rationale += gen_cot_rationale(a+b, P, mode="subtract")
    else:
        rationale += f"\n{a+b}<{P}\n{a+b}"
    return rationale

In [40]:
print(gen_mod_add_cot_rationale(102, 98, 113))

102+98,,C:0
10+9,0,C:1
1+,00,C:1
+,200,C:0
200
200>=113
200-113,,C:0
20-11,7,C:-1
2-1,87,C:-1
-,087,C:0
87


In [41]:
for a in range(113,226):
    gen_cot_rationale(a,113)